In [ ]:
import os, argparse, datetime, logging, traceback, time
import time, random, warnings
import pandas as pd
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver

def main(IdDaysDict, verbose=False):
    ### Output
    col = 'StockID, StockName, Date, KRaw, DRaw, JRaw, K, D, J'
    dfStockKDJ = pd.DataFrame(columns = col.split(', '))

    ### XpathSetting ###
    # XpathSettingButton ='/html/body/div[23]/input'
    # XpathIdxLengthInput = '/html/body/div[21]/table/tbody/tr[3]/td/table[1]/tbody/tr[2]/td[1]/table/tbody/tr/td[1]/table/tbody/tr[4]/td[2]/input'
    # XpathShiftInput = '/html/body/div[21]/table/tbody/tr[3]/td/table[1]/tbody/tr[2]/td[1]/table/tbody/tr/td[1]/table/tbody/tr[10]/td[2]/input'
    # XpathSettingConfirmButton = f'/html/body/div[24]/table/tbody/tr[3]/td/table[2]/tbody/tr[2]/td[1]/button'
    # XpathPreviousDayButton = '/html/body/div[20]/nobr/table/tbody/tr/td[2]/nobr/input[2]'
    XpathAnchor = '/html/body/table[2]/tbody/tr/td[3]/table[2]/tbody/tr[2]/td/canvas'
    XpathStockName = '/html/body/table[2]/tbody/tr/td[3]/table[2]/tbody/tr[1]/td/table/tbody/tr/td[2]/nobr/b'
    # XpathDate = '/html/body/div[3]/table/tbody/tr/td/div[1]/table/tbody/tr/td[7]'                           # OldVersion.~20221122
    # XpathK = '/html/body/div[9]/table/tbody/tr/td[1]/nobr[1]/font'
    # XpathD = '/html/body/div[9]/table/tbody/tr/td[1]/nobr[2]/font'
    # XpathJ = '/html/body/div[9]/table/tbody/tr/td[1]/nobr[3]/font'

    ### webdriver
    if verbose: print('Opening Webdirver...')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--start-maximized')
    # chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                              options=chrome_options)
    driver.maximize_window()
    FirstSetting = True
    ErrorMessage = []
    progress = IdDaysDict.items() if verbose else tqdm(IdDaysDict.items())
    for STOCK_ID, DAY in progress:
        if verbose:print(f'Fetching  StockID  :  {STOCK_ID}  information ...\t(1/{DAY})\t', end = '')
        url = f'https://goodinfo.tw/tw/ShowK_Chart.asp?STOCK_ID={STOCK_ID}&CHT_CAT2=DATE'
        driver.get(url)
        # OldVersion
            # settingButton = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'StockCanvas_btnConfig')))
        try:
            settingButton = driver.find_element(By.ID, 'StockCanvas_btnConfig')
            settingButton.click()
        except Exception as e:
            print(f'There is no data available for this StockID : {STOCK_ID}')
            ErrorMessage.append(e)
            continue

        if FirstSetting:
            idxLengthInput = driver.find_element(By.ID, 'CFG_InitIndexLength')
            shiftInput = driver.find_element(By.ID, 'CFG_CtrlShiftCnt')
            for n in range(20,25):
                try:
                    XpathSettingConfirmButton = f'/html/body/div[{n}]/table/tbody/tr[3]/td/table[2]/tbody/tr[2]/td[1]/button'
                    settingConfirmButton = driver.find_element(By.XPATH, XpathSettingConfirmButton)
                except:
                    pass
            idxLengthInput.send_keys('\b\b\b1')
            shiftInput.send_keys('\b\b1')
            driver.execute_script("arguments[0].scrollIntoView(true);",
                                  driver.find_element(By.XPATH, XpathAnchor))
            settingConfirmButton.click()
        else:
            driver.execute_script("arguments[0].scrollIntoView(true);",
                                  driver.find_element(By.XPATH, XpathAnchor))

        for i in range(DAY):
            stockName = driver.find_element(By.XPATH, XpathStockName).text.split()[1]
            
            for n in range(3,5):
                try:
                    XpathDate = f'/html/body/div[{n}]/table/tbody/tr/td/div[1]/table/tbody/tr/td[7]'
                    date = driver.find_element(By.XPATH, XpathDate).text.strip()
                except:
                    pass
            
            for n in range(9,15):
                try:
                    XpathK = f'/html/body/div[{n}]/table/tbody/tr/td[3]/nobr[1]/font'
                    XpathD = f'/html/body/div[{n}]/table/tbody/tr/td[3]/nobr[2]/font'
                    XpathJ = f'/html/body/div[{n}]/table/tbody/tr/td[3]/nobr[3]/font'
                    KRaw, DRaw, JRaw = map(lambda x: driver.find_element(By.XPATH, x).text[3:], (XpathK, XpathD, XpathJ))
                except:
                    pass

            K, D, J = map(lambda x: float(x[:-1]), (KRaw, DRaw, JRaw))
            dfStockKDJ.loc[len(dfStockKDJ)] = tuple([STOCK_ID, stockName, date, KRaw, DRaw, JRaw, K, D, J])
            if verbose: print(f'|| {STOCK_ID} | {stockName} | {date} | K: {KRaw:^6s} | D: {DRaw:^6s} | J: {JRaw:^6s} ||')
            if i == DAY - 1: break # early break

            # change date
            if verbose: print(f"Fetching the previous day's information ...\t({i+2}/{DAY})\t", end = '')
            previousDayButton = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'btnCtrlShiftLeft')))
            previousDayButton.click()
            time.sleep(1)
        if verbose: print()
    driver.close()
    if verbose:
        print('All query is done!!')
        print('There is some Error occured, please check!')
        for e in ErrorMessage:
            print(e)
    return dfStockKDJ


In [ ]:
### Example
# Input
IdDaysDict = {'0050': 1,
              '3443': 1}

df = main(IdDaysDict)
display(df)